In [21]:
from IPython.core.debugger import set_trace

import numpy as np
import scipy as sp
import scipy.optimize as optim

import gdual as gd
import forward as fwd

import matplotlib.pyplot as plt

In [55]:
# construct a poisson PGF by matching moments with some dist'n p 
# if provided, renormalize the PGF w/ Z
def APGF_Poiss(mean_p, Z = 1.0):
    lambda_q = mean_p
    theta_q  = [lambda_q]
    
    return lambda s: Z * fwd.poisson_pgf(s, theta_q)

# construct a NB PGF by matching moments with some dist'n p 
# if provided, renormalize the PGF w/ Z
def APGF_NB(mean_p, var_p, Z = 1.0):
    r_q = (mean_p ** 2) / (var_p - mean_p)
    p_q = 1 - (mean_p / var_p)
    theta_q = [r_q, p_q]
    
    return lambda s: Z * fwd.negbin_pgf(s, theta_q)

def APGF(F_p):
    # compute the normalization constant for F
    Z = F_p(1)
        
    if isinstance(Z, gd.GDual) or isinstance(Z, gd.LSGDual):
        Z = Z.unwrap_coefs(Z.coefs)[0]
    
    # renormalize the distribution before computing moments
    F_star = lambda s: F_p(s) / Z
    
    # construct M_p, the MGF of p, from F_star
    M_p = lambda t: F_star(np.exp(t))

    # use the MGF to compute the first k moments of p
    k = 2
    t0_gd = gd.GDual(0, q = k)
    moments_p = M_p(t0_gd)
    moments_p = moments_p.unwrap_coefs(moments_p.coefs, as_log = False)[1:] * sp.misc.factorial(np.arange(1, k+1))
    
    mean_p = moments_p[0]
    var_p  = moments_p[1] - (moments_p[0] ** 2)
    
    if mean_p == var_p:
        return APGF_Poiss(mean_p, Z), Z
    else:
        return APGF_NB(mean_p, var_p, Z), Z

In [76]:
# GDualType = gd.LSGDual

def APGF_Forward(y,
                 immigration_pgf,
                 theta_immigration,
                 offspring_pgf,
                 theta_offspring,
                 rho,
                 GDualType = gd.LSGDual,
                 d = 0):
    def Gamma_k(u_k, Alpha_PGF, k):
        #F(.) = offspring_pgf(u_k, theta_offspring[k-1])
        #G(.) = immigration_pgf(u_k, theta_immigration[k])
#         print(offspring_pgf(u_k, theta_offspring[k-1]))
        res = Alpha_PGF(offspring_pgf(u_k, theta_offspring[k-1])) * immigration_pgf(u_k, theta_immigration[k])
        return res
    
    def Alpha_k(s_k, Gamma_PGF, k):
        const = (s_k * rho[k])**y[k] / sp.misc.factorial(y[k])
        res = const * fwd.diff(Gamma_PGF, s_k*(1 - rho[k]), y[k], GDualType=GDualType)
        return res
    
    K = len(y)
    
    Gamma_PGFs = [None] * K
    Alpha_PGFs = [None] * (K+1)
    Z = [None] * K
    Alpha_PGFs[0] = lambda s_k: 1
    
    for i in range(K):
        Gamma = lambda u_k,k=i: Gamma_k(u_k, Alpha_PGFs[k], k)
        res = APGF(Gamma)
        Gamma_PGFs[i] = res[0]
        Z[i] = res[1]
        
        Alpha_PGFs[i+1] = lambda s_k,k=i: Alpha_k(s_k, Gamma_PGFs[k], k)
    
    if d == 0:
        alpha = Alpha_PGFs[-1]( 1.0 )
        logZ = np.log(alpha)
    else:
        alpha = Alpha_PGFs[-1]( GDualType(1.0, d) )
        logZ = alpha.get(0, as_log = True)
    
    return logZ, alpha, Alpha_PGFs

y     = np.array([3,2,4])
lmbda = np.array([5,5,5]).reshape(-1,1)
delta = np.array([0.4, 0.4]).reshape(-1,1)
rho   = np.array([0.25, 0.25, 0.25])

res = APGF_Forward(y,
                   fwd.poisson_pgf,
                   lmbda,
                   fwd.bernoulli_pgf,
                   delta,
                   rho)

# print(res[0][-1](1))
print(res)

res2 = fwd.forward(y,
                   fwd.poisson_pgf,
                   lmbda,
                   fwd.bernoulli_pgf,
                   delta,
                   rho)
print(res2)

# print(np.log(res[-1](1)))

(array([-inf]), array([ 0.]), [<function APGF_Forward.<locals>.<lambda> at 0x115ef7620>, <function APGF_Forward.<locals>.<lambda> at 0x115ef7158>, <function APGF_Forward.<locals>.<lambda> at 0x115ef78c8>, <function APGF_Forward.<locals>.<lambda> at 0x115ef7378>])
(-6.0515733053769445, array([ 0.00235416]), <function forward.<locals>.marginals at 0x115dfb0d0>)


/Users/kwinner/Work/latentcountmodels/python/forward.py:9: RuntimeWarning: divide by zero encountered in exp
  return np.exp(lmbda * (s - 1))
/Users/kwinner/Work/latentcountmodels/python/gdual_impl.py:222: RuntimeWarning: invalid value encountered in log
  out[0] = np.log(F[0])
/Users/kwinner/Work/latentcountmodels/python/gdual_impl.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  out[0] = 1. / F[0]
/Users/kwinner/Work/latentcountmodels/python/gdual_impl.py:244: RuntimeWarning: divide by zero encountered in double_scalars
  out[i] = 1. / F[0] * (-np.sum(out[:i] * F[i:0:-1], axis=0))
/Users/kwinner/Work/latentcountmodels/python/gdual_impl.py:244: RuntimeWarning: invalid value encountered in multiply
  out[i] = 1. / F[0] * (-np.sum(out[:i] * F[i:0:-1], axis=0))
/Users/kwinner/Work/latentcountmodels/python/forward.py:22: RuntimeWarning: invalid value encountered in multiply
  return ((p / (1 - ((1 - p) * s))) ** r)
/Users/kwinner/Work/latentcountmodels/python/forward

### Tests

In [47]:
## test method of computing moments

theta_p = [7.0, 0.5]
F_p = lambda s: fwd.negbin_pgf(s, theta_p)

# theta_p = [6.0]
# F_p = lambda s: fwd.poisson_pgf(s, theta_p)

# construct M_p, the MGF of p
M_p = lambda t: F_p(np.exp(t))

# use the MGF to compute the first k moments of p
k = 2
t0_gd = gd.GDual(0, q = k)
moments_p = M_p(t0_gd)
moments_p = moments_p.unwrap_coefs(moments_p.coefs, as_log = False)[1:] * sp.misc.factorial(np.arange(1, k+1))

print(moments_p)

mu_p  = moments_p[0]
var_p = moments_p[1] - (moments_p[0] ** 2)

print(mu_p)
print(var_p)

r_q = (mu_p ** 2) / (var_p - mu_p)
p_q = 1 - (mu_p/var_p)

print(r_q)
print(p_q)

[  7.  63.]
7.0
14.0
7.0
0.5


### OLD CODE

In [11]:
# ## OLD VERSIONS OF APGF WHICH DIDN'T RENORMALIZE BEFORE TAKING MOMENTS

# def APGF_Poiss_1(mean_p):
#     lambda_q = mean_p
#     theta_q  = [lambda_q]
    
#     return lambda s: fwd.poisson_pgf(s, theta_q)

# def APGF_NB_1(mean_p, var_p):
#     r_q = (mean_p ** 2) / (var_p - mean_p)
#     p_q = 1 - (mean_p / var_p)
#     theta_q = [r_q, p_q]
    
#     return lambda s: fwd.negbin_pgf(s, theta_q)

# def APGF_1(F_p):
#     Z = F_p(1)
    
#     # construct M_p, the MGF of p
#     M_p = lambda t: F_p(np.exp(t))

#     # use the MGF to compute the first k moments of p
#     k = 2
#     t0_gd = gd.GDual(0, q = k)
#     moments_p = M_p(t0_gd)
#     moments_p = moments_p.unwrap_coefs(moments_p.coefs, as_log = False)[1:] * sp.misc.factorial(np.arange(1, k+1))
    
#     mean_p = moments_p[0]
#     var_p  = moments_p[1] - (moments_p[0] ** 2)
    
#     if mean_p == var_p:
#         return APGF_Poiss_1(mean_p), Z
#     else:
#         return APGF_NB_1(mean_p, var_p), Z
    
# ## test methods for APGF

# theta_p = [7.0, 0.5]
# F_p = lambda s: 0.4 * fwd.negbin_pgf(s, theta_p)

# print(F_p(0))

# Fhat_p_1 = APGF(F_p)[0]
# print(Fhat_p_1(0))

# Fhat_p_2 = APGF_1(F_p)[0]
# print(Fhat_p_2(0))

In [72]:
### SCRAP TESTING AREA

# t0_gd.unwrap_coefs(t0_gd.coefs)[0]

np.log(res[0][1](1))

# r = [f(1) for f in res[0]]
# r

# res[0][1](1)

/Users/kwinner/Work/latentcountmodels/python/forward.py:9: RuntimeWarning: divide by zero encountered in exp
  return np.exp(lmbda * (s - 1))
/Users/kwinner/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([-2.37232882])